In [1]:
import gspread
import pandas as pd
import requests
import time
from thefuzz import process  # Biblioteca para fuzzy matching
from oauth2client.service_account import ServiceAccountCredentials
from unidecode import unidecode

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Autenticação com credenciais do JSON (substitua pelo seu arquivo JSON)
creds = ServiceAccountCredentials.from_json_keyfile_name("../credenciais/credenciais.json", scope)
client = gspread.authorize(creds)
spreadsheet = client.open('TPV Final')

In [124]:
def get_tpv_database(bank):
    sheet = spreadsheet.worksheet(f"Base ({bank})")  # Substitua pelo nome da aba da planilha
    data = sheet.get_all_values()
    if bank == 'inter':
        i = 3
    elif bank == 'own': 
        i = 2
    headers = data[i]  # A terceira linha (índice 2) contém os cabeçalhos
    db = pd.DataFrame(data[i+1:], columns=headers)
    #colunas_desejadas = ["Nome Completo"] + [col for col in db.columns if "/" in col]
    return db

In [128]:
get_tpv_database('own')

,Nº,CNPJ Estabelecimento,Nome Completo,Nome Fantasia,Dias Transacionados,ACM,17/07/2024,18/07/2024,19/07/2024,22/07/2024,...,14/03/2025,15/03/2025,16/03/2025,17/03/2025,18/03/2025,19/03/2025,20/03/2025,21/03/2025,22/03/2025,23/03/2025
0,1,667267352,MARAISA DA SILVA CABRAL,MARAISA DA SILVA CAB,8,"217,50",,,,,...,,,,,,,,,,
1,2,708549632,JOAO GOMES DA SILVA,JOAO GOMES DA SILVA,26,"1.420,37",,,,,...,,,,,,,,,,
2,3,724945300,CLAUDIONOR CUNHA SANTOS,KLAU STUDIO HAIR,1,"5,00",,,,,...,,,,,,,,,,
3,4,870409301,CLEONILDE SANTOS DE MATOS,CLEONILDE SANTOS DE,4,"307,31",,,,,...,,,,,,,,,,
4,5,902091417,JUAREZA PATRICIA SOUZA FERNANDES ALVES,LANCHONETE CANTINHO,3,"24,08",,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,190,85983357115,FABIO EUSTAQUIO GUALBERTO SOARES SILVA,FABIO EUSTAQUIO GUAL,9,"399,00",,,,,...,,,,,,,,,,
190,191,90079826806,NICOLAS COLIN CORREIA,NICOLAS COLIN CORREI,3,"5,90",,,,,...,,,,,,,,,,
191,192,95780254087,FABIANA DA SILVA,FABIANA DA SILVA,2,"3,00",,,,,...,,,,,,,,,,
192,193,98012924749,ROGERIO LUIZ DA SILVA,ROGERIO LUIZ DA SILV,4,"145,10",,,,,...,,,,,,,,,,


# UPDATING DATABASE

In [92]:
bank = 'own'
sheet = spreadsheet.worksheet(f"Base ({bank})")  # Substitua pelo nome da aba da planilha
data = sheet.get_all_values()
if bank == 'inter':
    i = 3
elif bank == 'own': 
    i = 2
headers = data[i]  # A terceira linha (índice 2) contém os cabeçalhos
db = pd.DataFrame(data[i+1:], columns=headers)

In [93]:
df = pd.read_csv('relatorio_diario_own.csv')

df["VGT"] = (
    df["VGT"]
    .str.replace("R$", "")   # Remove "R$"
    .str.replace(".", "")    # Remove pontos dos milhares
    .str.replace(",", ".")   # Substitui vírgula decimal por ponto
).astype(float)

df['CNPJ Estabelecimento'] = df['CNPJ Estabelecimento'].astype(str)

data_metrica = df['Data Métrica'][0]
df_grouped = df.groupby("CNPJ Estabelecimento", as_index=False)["VGT"].sum()

In [102]:
df_grouped = df_grouped.rename(columns={"VGT": data_metrica})


In [114]:
df_grouped

,CNPJ Estabelecimento,10/03/2025
0,49820887000153,18.0
1,53758067000156,7.0
2,55507385000170,2.5
3,6305354421,700.0
4,6702705476,20.5
5,7733325706,27.5
6,968362532,852.0
7,98012924749,139.1


In [112]:
date_col = df_grouped.columns[1]
# Atualizar os valores da coluna correspondente à data
for index, row in df_grouped.iterrows():
    cnpj = row.iloc[0]
    value = row.iloc[1]

    # Verifica se o CNPJ existe no DataFrame principal antes de atualizar
    mask = db["CNPJ Estabelecimento"] == cnpj
    if mask.any():
        db.loc[mask, date_col] = value  # Atualiza os valores correspondentes

In [123]:
db[['Nome Completo', date_col]][date_col].value_counts()

10/03/2025
         186
852.0      1
700.0      1
20.5       1
27.5       1
18.0       1
7.0        1
2.5        1
139.1      1
Name: count, dtype: int64

In [127]:
# Converter DataFrame para lista (sem cabeçalhos)
data_to_update = db.values.tolist()


In [135]:
sheet.update(range_name="A4", values=data_to_update)


{'spreadsheetId': '1By0bJacAhBjZaELBw6vAFqaNd4B3gJt5JJKiLsswlm8',
 'updatedRange': "'Cópia de Base (own)'!A4:IR197",
 'updatedRows': 194,
 'updatedColumns': 252,
 'updatedCells': 48888}

# UPDATING CRM (AGENDOR)

##### Getting full database

In [9]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd

In [25]:
gauth = GoogleAuth()


#gauth.settings['client_config_file'] = '../credenciais/credenciais_drive.json'

# Tenta carregar as credenciais salvas ou autentica via servidor web local
gauth.LocalWebserverAuth()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=669886130571-7i6v3tb4g14m9jui26rfjeod3mlf208p.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [61]:
import pandas as pd
import requests
from io import BytesIO

# 📌 URL para exportar a planilha como Excel (.xlsx)
xlsx_url = "https://docs.google.com/spreadsheets/d/1GpB1ed23HZhNKQciU3DSKC05ZowEirr0/export?format=xlsx&gid=520903099"

# 🔽 Baixar o arquivo
response = requests.get(xlsx_url)
response.raise_for_status()  # Garante que o download foi bem-sucedido

# 📖 Ler o arquivo Excel diretamente da memória
df_inter = pd.read_excel(BytesIO(response.content), engine="openpyxl", sheet_name = 'Base (inter)', header=3)

# 📌 URL para exportar a planilha como Excel (.xlsx)
xlsx_url = "https://docs.google.com/spreadsheets/d/1GpB1ed23HZhNKQciU3DSKC05ZowEirr0/edit?usp=sharing&ouid=105417421490865173338&rtpof=true&sd=true"

# 🔽 Baixar o arquivo
response = requests.get(xlsx_url)
response.raise_for_status()  # Garante que o download foi bem-sucedido

# 📖 Ler o arquivo Excel diretamente da memória
df_own = pd.read_excel(BytesIO(response.content), engine="openpyxl", sheet_name = 'Base (own)', header=2)

BadZipFile: File is not a zip file

In [27]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd



# 📥 ID da planilha no Google Drive
file_id = "1GpB1ed23HZhNKQciU3DSKC05ZowEirr0"
output_file = "planilha.xlsx"
drive = GoogleDrive(gauth)

# 📌 Baixar o arquivo
file = drive.CreateFile({'id': file_id})
file.GetContentFile(output_file)

# 📖 Ler a planilha no Pandas
df = pd.read_excel(output_file, sheet_name="Base (inter)")

TypeError: expected <class 'int'>

In [182]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd

# 📌 Autenticar com o Google Drive
gauth = GoogleAuth()

# Carrega as credenciais do arquivo credentials.json
gauth.LoadCredentialsFile("../credenciais/credenciais.json")

KeyError: '_module'

In [362]:
df_final = (
    pd.concat([data_own, data_inter], ignore_index=True)
    .fillna(0)
    .melt(id_vars=["Nome Completo"], var_name="date", value_name="value")  # Equivalente ao gather()
    .assign(
        value=lambda d: d["value"].fillna("0")  # Substitui NA por '0'
        .astype(str)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
        .replace("", "0")
        .astype(float),  # Converte valores para float
        date=lambda d: pd.to_datetime(d["date"], format="%d/%m/%Y"),  # Converte data
    )
    .rename(columns={"Nome Completo": "name"})  # Renomeia a coluna
    .sort_values(by=["name", "date"])  # Ordena os dados
    .reset_index(drop=True)
    .assign(
        tpv_60_d=lambda d: d.groupby("name")["value"].transform(lambda x: x.rolling(window=60, min_periods=1).sum()),
        tpv_30_d=lambda d: d.groupby("name")["value"].transform(lambda x: x.rolling(window=30, min_periods=1).sum()),
        tpv_historico=lambda d: d.groupby("name")["value"].cumsum(),
    )
    .loc[lambda d: d['date'] == pd.Timestamp.today().normalize()]
    .drop(columns=['date', 'value'])
)

In [590]:
API_KEY = "881078-ec5446d8-7fbd-4fac-806d-8a4d81eece36"
URL = "https://api.agendor.com.br/v3/deals"


headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}

In [728]:
# Função para realizar o scraping
def fetch_data():
    next_url = URL
    params = {"per_page": 100}
    all_json_deals_data = []
    wait_time = 10  # Tempo inicial de espera em segundos

    while next_url:
        try:
            response = requests.get(next_url, headers=headers, params=params)

            if response.status_code == 200:
                json_deal_data = response.json().get('data', [])
                all_json_deals_data.extend(json_deal_data)
                links = response.json().get('links', {})
                next_url = links.get('next', False)
                wait_time = 10  # Reseta o tempo de espera se der certo

            elif response.status_code == 429:
                print(f"Erro 429: Muitas requisições. Aguardando {wait_time} segundos antes de tentar novamente...")
                time.sleep(wait_time)
                wait_time *= 2  # Dobra o tempo de espera a cada falha

            else:
                print(f"Erro {response.status_code}: {response.text}")
                break  # Sai do loop se for outro erro

        except requests.exceptions.JSONDecodeError:
            print("Erro ao decodificar JSON. Resposta vazia ou malformada.")
            break
        except requests.exceptions.RequestException as e:
            print(f"Erro na requisição: {e}")
            break

    return all_json_deals_data

In [730]:
deals = fetch_data()

In [ ]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Autenticação com credenciais do JSON (substitua pelo seu arquivo JSON)
creds = ServiceAccountCredentials.from_json_keyfile_name("../credenciais/credenciais.json", scope)
client = gspread.authorize(creds)
spreadsheet = client.open('TPV Final')

In [732]:
sheet = spreadsheet.worksheet("02. Own")  # Substitua pelo nome da aba da planilha

data = sheet.get_all_values()

# Definir os cabeçalhos como a terceira linha
headers = data[4]  # A terceira linha (índice 2) contém os cabeçalhos

# Criar um DataFrame a partir da quarta linha em diante
db_names = pd.DataFrame(data[5:], columns=headers)[['PESSOA RELACIONADA', 'CADASTRO OWN']]

In [734]:
mapping = dict(zip(db_names["CADASTRO OWN"], db_names["PESSOA RELACIONADA"]))

In [736]:
df_final["name"] = df_final["name"].map(lambda x: mapping.get(x, x)).apply(normalize_name)
df_final['name'] = df_final['name'].apply(lambda x: 'fernanda sousa vieira' if x == 'fernada sousa vieira' else x)

In [738]:
all_crm_names = {}
for deal in deals:
    if deal['person'] and deal['dealStatus']['name'] == 'Em andamento':
        crm_name = normalize_name(deal['person']['name'])
        if crm_name not in all_crm_names:
            all_crm_names[crm_name] = deal['id']

In [740]:
def normalize_name(name):
    return unidecode(name).lower().strip()


In [742]:
matches = {}

# Loop para encontrar o nome mais próximo no CRM
for name in df_final["name"]:
    name_parts = name.split()
    first_name = name_parts[0]  # Pegamos o primeiro nome
    score_for_name = {}

    for crm_name in all_crm_names.keys():
        crm_name_parts = crm_name.split()
        if crm_name_parts:
            first_crm_name = crm_name_parts[0]

            # 🔍 Se o primeiro nome for igual, analisamos os sobrenomes
            if first_name == first_crm_name:
                score = sum(1 for surname in name_parts[1:] if surname in crm_name_parts)
                score_for_name[crm_name] = score  # Armazena o score

    # Se encontramos pelo menos um nome com o primeiro nome igual
    if score_for_name:
        best_match = max(score_for_name, key=score_for_name.get)  # Pega o nome com maior score
        matches[name] = all_crm_names[best_match]  # Mapeia para o CRM
    else:
        print(name)
        matches[name] = None  # Nenhuma correspondência encontrada


alice alves rosostolato
maicon carvalho de oliveira
elisangela aparecida silva de souza
misael rodrigues araujo
erick jesus gotardo
elcio de araujo valente
franclin jonas vieira da silva
jonas freitas da hora
marivaldo da silva cavalcante


In [770]:
df_final["deal_id"] = df_final["name"].apply(matches.get)

In [788]:
df_final.dropna(inplace=True)

In [790]:
BASE_URL = "https://api.agendor.com.br/v3"

headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}

for i, row in df_final.iterrows():
    deal_id = str(int(row['deal_id']))
    tpv_historico = str(row['tpv_historico'])
    tpv_30_d = str(row['tpv_30_d'])
    tpv_60_d = str(row['tpv_60_d'])
    url = f"{BASE_URL}/deals/{deal_id}"
    payload = {"customFields": {'tpv_historico': tpv_historico, 'tpv_mes_anterior': tpv_30_d, 'tpv_m-2': tpv_60_d}}  

    wait_time = 10  # Tempo de espera inicial
    
    while True:
        response = requests.put(url, headers=headers, json=payload)

        if response.status_code == 200:
            wait_time = 10  # Reseta o tempo de espera se der certo
            print(f"Sucesso para {i}")  # Corrigido o uso de f-string
            break
            
        elif response.status_code == 429:
            print(f"Erro 429: Muitas requisições. Aguardando {wait_time} segundos antes de tentar novamente...")
            time.sleep(wait_time)
            wait_time *= 2  # Dobra o tempo de espera a cada falha
        else:
            print(f"Erro inesperado para {i}: {response.status_code}")
            break  # Você pode optar por parar ou continuar com outro comportamento

    print(f"Processado {i} de {len(df_final)}")
 

Sucesso para 0
Processado 0 de 225
Sucesso para 1
Processado 1 de 225
Sucesso para 2
Processado 2 de 225
Sucesso para 3
Processado 3 de 225
Sucesso para 4
Processado 4 de 225
Sucesso para 5
Processado 5 de 225
Sucesso para 6
Processado 6 de 225
Sucesso para 7
Processado 7 de 225
Sucesso para 8
Processado 8 de 225
Sucesso para 9
Processado 9 de 225
Sucesso para 10
Processado 10 de 225
Sucesso para 11
Processado 11 de 225
Sucesso para 12
Processado 12 de 225
Sucesso para 13
Processado 13 de 225
Sucesso para 15
Processado 15 de 225
Sucesso para 17
Processado 17 de 225
Sucesso para 18
Processado 18 de 225
Sucesso para 19
Processado 19 de 225
Sucesso para 20
Processado 20 de 225
Sucesso para 21
Processado 21 de 225
Sucesso para 22
Processado 22 de 225
Sucesso para 23
Processado 23 de 225
Sucesso para 24
Processado 24 de 225
Sucesso para 25
Processado 25 de 225
Sucesso para 26
Processado 26 de 225
Sucesso para 27
Processado 27 de 225
Sucesso para 28
Processado 28 de 225
Sucesso para 29
Proce

In [726]:
df_final.to_excel('df_final.xlsx')

In [598]:
import time
payload = {"dealStatusText": "lost"}  # Atualizando para "lost"
BASE_URL = "https://api.agendor.com.br/v3"

i = 0
for deal_id in bad_deals_id[88:]:
    url = f"{BASE_URL}/deals/{deal_id}/status"
    wait_time = 10  # Tempo de espera inicial

    while True:
        response = requests.put(url, headers=headers, json=payload)

        if response.status_code == 200:
            wait_time = 10  # Reseta o tempo de espera se der certo
            print(f"Sucesso para {i}")  # Corrigido o uso de f-string
            break
            
        elif response.status_code == 429:
            print(f"Erro 429: Muitas requisições. Aguardando {wait_time} segundos antes de tentar novamente...")
            time.sleep(wait_time)
            wait_time *= 2  # Dobra o tempo de espera a cada falha
        else:
            print(f"Erro inesperado para {i}: {response.status_code}")
            break  # Você pode optar por parar ou continuar com outro comportamento

    i += 1
    print(f"Processado {i} de {len(bad_deals_id)}")

{'data': [{'id': 27531374,
   'parentDealId': None,
   'accountId': 698720,
   'organization': None,
   'person': {'id': 55328371,
    'name': 'Israel Elihom Da Silva Pereira',
    'email': None},
   'author': {'accountId': 698720, 'id': 860639, 'name': 'Guilherme Hely'},
   'dealStage': {'id': 3270603,
    'name': '4.2 ACOMPANHAR POSTAGEM',
    'sequence': 2,
    'funnel': {'id': 791207, 'name': '4 RECOLHIMENTO DE POS'}},
   'dealStatus': {'id': 1, 'name': 'Em andamento'},
   'lossReason': None,
   'owner': {'accountId': 698720, 'id': 863021, 'name': 'Juliane Sales'},
   'title': 'Israel Elihom Da Silva Pereira',
   'description': None,
   'startTime': '2024-11-07T00:00:00.000Z',
   'endTime': None,
   'wonAt': None,
   'lostAt': None,
   'createdAt': '2024-11-07T21:38:33.000Z',
   'updatedAt': '2025-03-13T17:20:00.000Z',
   'ranking': 0,
   'value': 0.0,
   'products': [],
   'allowedUsers': [{'accountId': 698720,
     'id': 860639,
     'name': 'Guilherme Hely'},
    {'accountId': 6

In [ ]:
214241.4